# Using `pandas` to explore firm accounting data

I've given you a list of firms from 2020 with variables
- "gvkey", "lpermno", "lpermno" = different datasets use different identifiers for firms
- "fyear" = the fiscal year the remaining variable apply to 
- "gsector" = gsector, an industry classification (see [the wiki article on GICS](https://en.wikipedia.org/wiki/Global_Industry_Classification_Standard))
- "state" = of headquarters
- "tic" = ticker
- various accounting statistics

This data is a small slice of Compustat, which is a professional grade dataset that contains accounting data from SEC filings. 

In [2]:
import pandas as pd
import numpy as np
from eda import insufficient_but_starting_eda
import pandas_datareader as pdr  # to install: !pip install pandas_datareader
from datetime import datetime

# this file can be found here: https://github.com/LeDataSciFi/ledatascifi-2021/tree/main/data
# if you click on the file, then "raw", you'll be at this url
# which contains the raw data 
# and pandas can download/load it without saving it locally!
url = 'https://raw.githubusercontent.com/LeDataSciFi/ledatascifi-2021/main/data/firms2020.csv'
firms_df = pd.read_csv(url) 

ModuleNotFoundError: No module named 'eda'

## Part 1 - create variables

Create three variables:
1. $log(sales)$
2. $leverage = (dlc+dltt)/at $
3. $capx\_ratio = capx/at$

Q1: Then `.describe()` these three variables right after creating them. (Don't do any cleaning yet! This is just a way for graders to check your definitions quickly.)

Insert cell(s) below this one as needed to finish this Part.

In [ ]:
# firms_df = 
import numpy as np
firms_df = (firms_df
            .assign(log_sale = np.log(firms_df['sale']), 
                    leverage = (firms_df['dlc']+firms_df['dltt'])/firms_df['at'],
                    capx_ratio = firms_df['capx'] / firms_df['at']
                   )            
           )

# will produce warning because obs with neg sales --> log(-#)


In a rigorous project, we would have to make decisions about how to deal with observations with negative sales. 

In this dataset, the typical solution researchers make is to drop observations with missing or negative assets/sales. 

In [ ]:
vars_to_desc = firms_df['log_sale','leverage','capx_ratio']
firms_df[vars_to_desc].describe().T # the .T transposes the table to fit on screen

## Part 2 - EDA and data cleaning

Q2: Implement the best practices from textbook section 3.2.5 for **the EDA dropdown and data cleaning dropdown.** 

Q3: After producing output, insert a markdown cell and summarize some possible issues you found. 

Note: You should **NOT** change any values here - you're just looking for issues to consider.

Insert cell(s) below this one as needed to finish this Part.


---------------

### Q2 - My work - EDA best practices

The code below only shows 10 rows (`head.()` + `tail()`). That's simply not enough. 

In addition to the below code, I looked at the dataset using

1. Random sampling of obs

    ```python
    firms_df[vars_to_desc].sample(50) # picks 50 random rows, rerun to see diff parts
    ```
    
2. Look at chunks - it's useful to see sequential rows

    ```python
    firms_df[vars_to_desc][115:135] # change the row numbers to see diff parts
    ```
    
3. Randomly pick N firms and pull all their observations:
    ```python
    list_of_firms = firms_df['tic'].unique() 
    rand_firms = np.random.choice(list_of_firms,5) 
    firms_df[vars_to_desc].query('tic in @rand_firms')
    ```
    
Discussion is below.   

In [ ]:
vars_to_desc = ['GVKEY','tic','fyear', # firm/yr ids
                'state','gsector',     # cat descriptors of firm
                'log_sale','leverage','capx_ratio', # our vars
                'sale','at','dlc','dltt','capx']    # underlying vars
insufficient_but_starting_eda(firms_df[vars_to_desc],['state','gsector','fyear'])

In [ ]:
print("There are",len(firms_df),"obs in the firm data",
      "covering",firms_df['tic'].nunique(),"unique tickers and ",
      firms_df['GVKEY'].nunique(),"unique GVKEYs."      
     )


### Q3 - Data summary and issues to think about

#### Info on the sample, the data's unit level

This dataset has accounting data on firms. It *appears* that GVKEY, LPERMNO, LPERMCO, and TIC are unique firm identifiers. There is also a firm name variable. 

Each firm has data for a given fiscal year, reported by FYEAR, and the reporting date is the `datadate` variable. 

**THE "UNIT" LEVEL FOR THIS DATASET IS FIRM YEAR even though the dataset we have only has 2020 observations.** 

#### Variables

1. There are 56 "states"?
2. Gsector is "number" but the numbers correspond to industry categories. It's a CATEGORICAL variable. 
3. Some observations have negative sales (and thus undefined logs). Will need to deal with that. 
4. Missing values - We have roughly 1230 observations with most variables. Should we restrict to these? But even in this subset, about 25% have no CAPX data. If those missing values should be zero, let's set it to zero. We should look into this...
    - I usually set R&D and CAPX to 0 if missing. 
    - Then I drop observations where this choice causes the sources/uses of cash flow identity ($0=Inv + Ch_Cash + Div - Ch_Eqty - Ch_Debt - CF$) to fail). Checking this is a bit involved, so I'm skipping it. 


## Part 3 - Subsample analysis

Let's explore how firms vary by state and industry. There are a lot of combinations of state-industry, so for simplicity let's only look at a few of them. 

So first **filter the data so you only have firms from California and Pennsylvania; and from the finance and industrial sectors**.

Q4: Produce the following statistics **for all four combinations of state-industry we're left with (CA-finance, CA-industry, PA-finance, and PA-industry):**
- the number of observations
- the number of unique firms (use a different function than `count` or `len`)
- the distribution of log(sales): mean, std, min, max



Q5: Which of the four state-industries has the largest firms on average, as measured by assets (the $at$ variable)?

Insert cell(s) below this one as needed to finish this Part.

---

### Q4 - My work

In [3]:
subsample = firms_df.query('state in ["CA","PA"] & gsector in ["40","20"]') 
# fun trick: display > print for prettyness of DFs

display(subsample.pivot_table(index='gsector',columns='state',
                   values='GVKEY',aggfunc=['count','nunique'])
     )


NameError: name 'firms_df' is not defined

The number of observations is the same as the number of firms because there is only one year of data per firm in this subsample. 

Let's look at the distribution of log sales. 

In [4]:
display(subsample
      .pivot_table(index='gsector',columns='state',
                   values='log_sale',aggfunc=['describe'])
      # this is tough to read... lets make it less wide:  
      .stack()
      # then reorder the columns  
      .droplevel(0, axis=1)
      .reindex(columns=['count','mean','sd','min','25%','50%','75%','max'] )
      # too many decimals! change the print format  
      .style.format("{:.2f}")
     )

NameError: name 'subsample' is not defined

### Q5 - My work

California finance (`gsector` = 40) firms are the largest on average in this subsample. 

In [5]:
subsample.pivot_table(index='gsector',columns='state',values='at').style.format("{:.2f}")

NameError: name 'subsample' is not defined

## Part 4 - Covid stock returns

For this part, use the firms in the subsample from part 3. 

Get their tickers from the data. Then copy relevant parts of  `stock_prices.ipynb` here and modify it so that
- it uses the firms in the subsample from part 3
- downloads stock prices from Feb 1 2020 to Apr 30 2020
- HINT: I was able to download prices for 57 firms. 

Then, compute daily returns for each firm.

Now we are ready for some questions: 
- Q6: Output `.describe()` for the return variable. 
    - HINT: put the returns for all firms in one variable (as opposed to having 57 return variables, one for each firm)
- Q7: Compute and report the average return each day for each state-industry (so for each day, you now have 4 returns - one for CA-finance firms, and so on...)
    - HINT: You'll need to merge the state and industry variables with the stock return data. 
    - HINT: `pd.merge()`
    - HINT: We will talk more about merging in future weeks, but [the top voted answer here is pretty good](https://stackoverflow.com/questions/53645882/pandas-merging-101).
- Q8: Compute the volatility for each firm, then take the average within each state-industry: Which industry-state had the lowest and highest average volatility? 
- Compute the **weekly** returns for each state-industry and answer 
    - Q9a: Which industry-state had the best single week in this time frame?
    - Q9b: Which industry-state had the worst single week in this time frame?
    - HINT: $r[M-F]=(1+r_M)*...*(1+r_F)-1$
- Q10: Plot the weekly returns for for each state-industry

Insert cell(s) below this one as needed to finish this Part.

---

### My work

Let's start by downloading the stock prices and reformatting the data to use.

In [6]:
import pandas_datareader as pdr  # to install: !pip install pandas_datareader
from datetime import datetime

start = datetime(2020, 2, 1)
end = datetime(2020, 4, 30)

# load
stock_prices = pdr.get_data_yahoo(subsample['tic'].to_list(), start=start, end=end)
stock_prices = stock_prices.filter(like='Adj Close') # reduce to just columns with this in the name
stock_prices.columns = subsample['tic'].to_list() # put their tickers as column names
stock_prices = stock_prices.stack().swaplevel().sort_index().reset_index()
stock_prices.columns = ['Firm','Date','Adj Close']
stock_prices['ret'] = stock_prices.groupby('Firm')['Adj Close'].pct_change()

# add state + gsector variables
stock_prices= stock_prices.merge(subsample[['tic','gsector','state']],left_on='Firm',right_on='tic')



NameError: name 'subsample' is not defined

### Q6 - `.describe()` returns in the sample

We've already formatted the data. Just output:

In [ ]:
stock_prices['ret'].describe()

### Q7 - Compute portfolio returns and average over sample period

First we need to compute the portfolio returns for each day. Here, our portfolios are equally weighted baskets of all the firms in a given state and industry.

In [ ]:
daily_port_ret = (stock_prices
                  .groupby(['state','gsector','Date']) # for each portfolio, for each day
                  ['ret'].mean()                       # avg the return for that day for the firms in the port
                  .reset_index()                       # you can work with state/sector/date as index or vars
                                                       # I decided to convert them to variables and sort                                                    
                  .sort_values(['state','gsector','Date'])
                 )



In [ ]:
# option 1:
daily_port_ret.groupby(['state','gsector'])['ret'].mean()

# option 2:
daily_port_ret.pivot_table(index='gsector',columns='state',values='ret')

# option 2+formatting
(daily_port_ret.pivot_table(index='gsector',columns='state',values='ret')
 .multiply(100) # these are now percentages!!!
.style.format("{:.3f}%")
)

### Q8 - California Industrials were most volatile over this period

Here I use the `std`, but the conclusion is the same with `var`.

In [1]:
(stock_prices
 # compute std for each firm
 # I added state/sector to the groupby JUST to keep those vars around
 .groupby(['state','gsector','tic'])['ret'].std()   
 # now, we have 3 index vars. groupby on index vars by adding "level=" inside groupby
 # and we have a single Series var (without a name) so we just take a mean
 .groupby(level=['state','gsector']).mean()
) 

(stock_prices
 
 .groupby(['state','gsector','tic'])['ret'].std()  .groupby(level=['state','gsector']).mean()
) 

NameError: name 'stock_prices' is not defined

### Q9 - Compute weekly returns for the portfolios

The biggest challenge is compounding the returns. 


#### How to compound returns

Let $r_t = (P_t+D_t)/P_{t-1}$ be the simple return for a single period with dividends included. 

Let $R_t = (1+r_t)$ be the **gross** (thus, the capital "R") return for a single period.

Then the gross return for a number of periods is

\begin{equation}
R[0,T] = \prod_{t=1}^T R_t
\end{equation}

and the return for the same set of periods is simply the above minus 1.

\begin{align}
r[0,T] = & R[0,T] - 1  \\
= & (\prod_{t=1}^T R_t) - 1
\end{align}

There is one more way to compute the gross return: "Sum the log returns, then exponentiate":

\begin{align}
R[0,T] = & \prod_{t=1}^T R_t \\
       = & \prod_{t=1}^T ( e(ln( R_t))        ) \\
       = & e(\sum_{t=1}^T ln(R_t)) \\
\end{align}
the second line works because $e(ln(x))=x$ and the last line works because $e^a*e^b*e^c=e^{a+b+c}$.






In [ ]:
weekly_port_ret = (daily_port_ret
                   # compute gross returns for each asset (and get the week var)
                   .assign(R = 1+daily_port_ret['ret'],
                           week = daily_port_ret['Date'].dt.isocalendar().week)
                   # for each portfolio and week...
                   .groupby(['state','gsector','week'])
                   # cumulate the returns
                   ['R'].prod()
                   # subtract one
                   -1
)



#### The best and worst weeks belonged to:

In [ ]:
print('\n\n-------------------------\nThe BEST week in this sample was:\n-------------------------')
print(weekly_port_ret.nlargest(1))
print('\n\n-------------------------\nThe WORST week in this sample was::\n-------------------------')
print(weekly_port_ret.nsmallest(1))

### Q10

In [36]:
# ugly but quick
weekly_port_ret
weekly_port_ret.unstack().T.plot()



NameError: name 'weekly_port_ret' is not defined